In [48]:
from langchain.chains import SequentialChain
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate
import requests
from fastapi import FastAPI
import os
from dotenv import load_dotenv
import tweepy
from instagram_private_api import Client
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

In [49]:
load_dotenv()

True

In [50]:
TWITTER_CREDS = {
    "api_key": os.getenv("TWITTER_API_KEY"),
    "api_secret": os.getenv("TWITTER_API_SECRET"),
    "access_token": os.getenv("TWITTER_ACCESS_TOKEN"),
    "access_secret": os.getenv("TWITTER_ACCESS_SECRET"),
}
INSTAGRAM_CREDS = {
    "username": os.getenv("INSTAGRAM_USERNAME"),
    "password": os.getenv("INSTAGRAM_PASSWORD")
}

In [ ]:
model_id = "/Users/saadkhalid/Documents/epita/s3/action_learning/cross-domain-recommender-movies-and-games/models/Llama-3.1-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.66s/it]
Some parameters are on the meta device because they were offloaded to the disk.


In [27]:
text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=500,
)

Device set to use mps


In [28]:
llm = HuggingFacePipeline(pipeline=text_gen_pipeline)

/var/folders/4l/1l38pkdx6hz8bl9tnjp6wpdm0000gn/T/ipykernel_89508/3493905580.py:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=text_gen_pipeline)


In [42]:
def prepare_rag():
    loader = TextLoader("data/crypto_banter_content.txt")
    documents = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings()
    vector_db = Chroma.from_documents(texts, embeddings)
    return vector_db.as_retriever()

In [43]:
prompt_template = PromptTemplate(
    input_variables=["trends", "rag_context"],
    template="""
    You are a virtual version of Crypto Banter. Generate social media content using their style.

    Recent crypto trends: {trends}
    Relevant context from past content:
    {rag_context}

    Generate a post that:
    1. Matches Crypto Banter's communication style
    2. Incorporates at least 2 trends
    3. Includes characteristic phrases (e.g., "DYOR", "To the moon!")
    4. Is under 280 characters
    """,
)

In [44]:
# Crypto trends processing with vetting
def get_vetted_trends():
    raw_trends = requests.get("https://api.coingecko.com/api/v3/search/trending").json()
    return [t["item"]["name"] for t in raw_trends["coins"] if is_relevant(t["item"])]

def is_relevant(coin):
    keywords = ["bitcoin", "ethereum", "defi", "nft", "web3", "altcoin"]
    return any(kw in coin["name"].lower() for kw in keywords)

# Content vetting and analysis
def vet_content(content):
    # Check for inappropriate language
    inappropriate_words = ["scam", "rug pull", "ponzi"]
    if any(word in content.lower() for word in inappropriate_words):
        return False, "Content contains inappropriate language."

    # Check for Crypto Banter's style
    style_keywords = ["DYOR", "To the moon", "HODL", "Altseason"]
    style_score = sum(1 for kw in style_keywords if kw in content)
    if style_score < 2:
        return False, "Content does not match Crypto Banter's style."

    # Check length
    if len(content) > 280:
        return False, "Content exceeds 280 characters."

    return True, "Content is approved."

# Social media posting functions
def post_to_twitter(content):
    auth = tweepy.OAuthHandler(TWITTER_CREDS["api_key"], TWITTER_CREDS["api_secret"])
    auth.set_access_token(TWITTER_CREDS["access_token"], TWITTER_CREDS["access_secret"])
    api = tweepy.API(auth)
    api.update_status(content)
    return "Tweet posted successfully"

def post_to_instagram(content):
    api = Client(INSTAGRAM_CREDS["username"], INSTAGRAM_CREDS["password"])
    api.post_photo(photo_path="crypto_image.jpg", caption=content)
    return "Instagram post successful"


In [47]:
# Main chain construction
retriever = prepare_rag()
app = FastAPI()

chain = SequentialChain(
    chains=[
        {"name": "get_trends", "func": get_vetted_trends, "output_key": "trends"},
        {
            "name": "retrieve_context",
            "func": lambda inputs: retriever.get_relevant_documents(inputs["trends"]),
            "output_key": "rag_context",
        },
        {
            "name": "generate_content",
            "func": lambda inputs: llm(prompt_template.format(**inputs)),
            "output_key": "content",
        },
        {
            "name": "vet_content",
            "func": lambda inputs: vet_content(inputs["content"]),
            "output_key": "vetting_result",
        },
        {
            "name": "post_content",
            "func": lambda inputs: {
                "twitter": post_to_twitter(inputs["content"]) if inputs["vetting_result"][0] else "Content not posted (vetting failed)",
                "instagram": post_to_instagram(inputs["content"]) if inputs["vetting_result"][0] else "Content not posted (vetting failed)",
            },
        },
    ],
    input_variables=[],
    output_variables=["content", "vetting_result"],
)


/var/folders/4l/1l38pkdx6hz8bl9tnjp6wpdm0000gn/T/ipykernel_89508/3554886173.py:8: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
No sentence-transformers model found with name sentence-transformers/all-mpnet-base-v2. Creating a new one with mean pooling.


OSError: sentence-transformers/all-mpnet-base-v2 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [46]:
!pip install sentence-transformers